<a href="https://colab.research.google.com/github/jaison-1920/RAG/blob/main/rag_with_opensource1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chat with website

### Installation

In [1]:
!pip install langchain sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# importing the libraries
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Chroma

from langchain.llms import HuggingFaceHub
from langchain.chains import RetrievalQA

In [3]:
import os
from getpass import getpass
HF_token = getpass()

··········


In [4]:
os.environ["HUGGINGFACEHUB_API_KEY"] = HF_token

In [5]:
URL = "https://tarunjain.netlify.app"

In [6]:
data=WebBaseLoader(URL)

In [7]:
content = data.load()

In [8]:
print(content)

[Document(page_content='\n\n\n\n\nTarun Jain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAbout Me\nEvents\nAchievements\nProjects\nWork\nConnect\n\nResume\n\n\nAI with Tarun\n\n\n\n\n\n\n\n\n\n\n\n\nIt\'s Me...  Tarun Jain\n\n Data Scientist @AI PlanetðŸ¥‘ ||  AI with Tarun- YouTubeðŸ¤—||  GDE in AI/MLðŸš€ ||  GSoC\'23 caMicroscope\nKnow more\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAbout Me\n\n\r\n                Hello! My name is Tarun R Jain. I\'m a passionate coder with expertise in Machine Learning, Image Processing, and Deep Learning. I\'ve published over 80 blog articles documenting my coding journey, and I\'m actively involved in various communities, including Hugging Face-Keras Working Group, Deep Learning AI- Bangalore Ambassador, TensorFlow User Group Bangalore- Assistant Organizer and Geeksforgeeks- Technical content writer.\r\n              \n\r\n                What I love most about coding is the ability to create something new and solve complex problems. I\'m constantly 

# chunking - text splitter

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256,chunk_overlap=0)

In [10]:
chunking = text_splitter.split_documents(content)

In [11]:
len(chunking)

32

In [12]:
chunking[0]

Document(page_content="Tarun Jain\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHome\nAbout Me\nEvents\nAchievements\nProjects\nWork\nConnect\n\nResume\n\n\nAI with Tarun\n\n\n\n\n\n\n\n\n\n\n\n\nIt's Me...  Tarun Jain", metadata={'source': 'https://tarunjain.netlify.app', 'title': 'Tarun Jain', 'language': 'pt-BR'})

In [13]:
chunking[3]

Document(page_content='communities, including Hugging Face-Keras Working Group, Deep Learning AI- Bangalore Ambassador, TensorFlow User Group Bangalore- Assistant Organizer and Geeksforgeeks- Technical content writer.', metadata={'source': 'https://tarunjain.netlify.app', 'title': 'Tarun Jain', 'language': 'pt-BR'})

#Embeddings

In [14]:
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key = HF_token,model_name="BAAI/bge-base-en-v1.5"
)

# Vector store

In [15]:
vector_store = Chroma.from_documents(chunking,embeddings)

# LLM model

In [46]:
model = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-beta",
                       model_kwargs={"temperature":0.5,"max_new_tokens":249,"max_length":64},
                       huggingfacehub_api_token=HF_token )

# Retriever

## step 1:Retrieval

In [27]:
retriever = vector_store.as_retriever(search_type="mmr",
                                      search_kwargs={"k":1})

In [28]:
query = "who is DevRel and AI Planet?"

In [29]:
docs_rel = retriever.get_relevant_documents(query)

In [30]:
docs_rel

[Document(page_content="Data Scientist @AI PlanetðŸ¥‘ ||  AI with Tarun- YouTubeðŸ¤—||  GDE in AI/MLðŸš€ ||  GSoC'23 caMicroscope\nKnow more\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAbout Me", metadata={'language': 'pt-BR', 'source': 'https://tarunjain.netlify.app', 'title': 'Tarun Jain'})]

## step 2: Augmentation

#### prompt

In [45]:
query = "Is Tarun a GDE in AI/ML?"

In [32]:
prompt = f"""
<|system|>
You are an AI assistant that follow instructions extremly well.
Please be truthful and give direct answers.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

## step 3: Generation

In [47]:
qa = RetrievalQA.from_chain_type(llm=model,chain_type="stuff",retriever=retriever)

In [48]:
response = qa(prompt)

In [54]:
print(response['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Appreciate my Work
Feel free to check this resources...






AI With Tarun:
Subscribe...



Blogs:
Medium: @jaintarun7
GeeksForGeeks: jaintarun
Analytics Vidya: tarun4641039



Newsletter
Subscribe to my Newsletter
















Connect

Question: 
<|system|>
You are an AI assistant that follow instructions extremly well.
Please be truthful and give direct answers.
</s>
<|user|>
Is Tarun a GDE in ML?
</s>
<|assistant|>

Helpful Answer:

I do not have access to real-time information or personal details about individuals. However, based on the information provided by tarun in his blogs, tarun has earned the google developer expert (gde) badge in machine learning (ml) in 2021. You can confirm this information by visiting the official gde website and searching for tarun's name in the ml category.
